**Stat 451 Final Project- Data Cleaning- Full and Ordinal**

*Dylan Asoh, Logan O'Brien, Samuel Peterson*

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv("train.csv")
# data_test = pd.read_csv("test.csv")
# data_test.head()

In [3]:
data_drop = data.drop(['host_has_profile_pic', 'host_identity_verified','thumbnail_url', 'first_review', 'last_review', 'host_response_rate', 'neighbourhood', 'description', 'latitude', 'longitude', 'host_since', 'id', 'name'], axis=1)
data_drop.columns

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'instant_bookable', 'number_of_reviews', 'review_scores_rating',
       'zipcode', 'bedrooms', 'beds'],
      dtype='object')

In [4]:
data_drop.isna().sum()

log_price                   0
property_type               0
room_type                   0
amenities                   0
accommodates                0
bathrooms                 200
bed_type                    0
cancellation_policy         0
cleaning_fee                0
city                        0
instant_bookable            0
number_of_reviews           0
review_scores_rating    16722
zipcode                   966
bedrooms                   91
beds                      131
dtype: int64

In [5]:
data_drop = data_drop.drop(['review_scores_rating', 'zipcode'], axis=1)
data_drop.columns

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'instant_bookable', 'number_of_reviews', 'bedrooms', 'beds'],
      dtype='object')

In [6]:
data_drop.isna().sum()

log_price                0
property_type            0
room_type                0
amenities                0
accommodates             0
bathrooms              200
bed_type                 0
cancellation_policy      0
cleaning_fee             0
city                     0
instant_bookable         0
number_of_reviews        0
bedrooms                91
beds                   131
dtype: int64

In [7]:
data_drop = data_drop.drop(11632)

In [8]:
data_clean_full = data_drop.copy()

In [9]:
row2=[]
final = []
amentity_totals_dict = {}
for row in data_clean_full['amenities']:
    row2 = (row.split(","))
    for item in row2:
        amenity = re.findall("[A-Za-z\s]+", item)
        if len(amenity) == 0:
            pass
        elif amenity[0] not in final:
            final.append(amenity[0])
            amentity_totals_dict[amenity[0]] = 1
        else:
            amentity_totals_dict[amenity[0]] += 1
def amentity_totals(listing):
    if len(listing) != 0:
        amenities = (listing.split(","))
        if "translation missing: en.hosting_amenity_50" in amenities:
            return len(amenities - 1)
        else:
            return len(amenities)
    else:
        return 0

data_clean_full["amenities"] = data_clean_full["amenities"].apply(amentity_totals)

In [10]:
def bed_type(bed):
    if bed == 'Real Bed':
        return 4
    elif bed == 'Pull-out Sofa':
        return 3
    elif bed == 'Futon':
        return 2
    elif bed == 'Couch':
        return 1
    else: #Airbed
        return 0

In [11]:
data_clean_full["bed_type"] = data_clean_full["bed_type"].apply(bed_type)

In [12]:
def cancellation_policy(policy):
    if policy == "flexible":
        return 3
    elif policy == "moderate":
        return 2
    elif policy =="strict":
        return 1
    else: # there are weird ambiguous "super" strict values that have _60 and _30 at end
        return 0

In [13]:
data_clean_full["cancellation_policy"].value_counts()

strict             32373
flexible           22545
moderate           19063
super_strict_30      112
super_strict_60       17
Name: cancellation_policy, dtype: int64

In [14]:
data_clean_full["cancellation_policy"] = data_clean_full["cancellation_policy"].apply(cancellation_policy)

In [15]:
def room_type(room):
    if room == "Entire home/apt":
        return 2
    elif room == "Private room":
        return 1
    else: #Shared Room
        return 0

In [16]:
data_clean_full["room_type"] = data_clean_full["room_type"].apply(room_type)

In [17]:
data_clean_full.loc[:,"cleaning_fee":]

,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,beds
0,True,NYC,f,2,1.0,1.0
1,True,NYC,t,6,3.0,3.0
2,True,NYC,t,10,1.0,3.0
3,True,SF,f,0,2.0,2.0
4,True,DC,t,4,0.0,1.0
...,...,...,...,...,...,...
74106,False,NYC,f,0,1.0,1.0
74107,True,LA,f,16,2.0,4.0
74108,True,NYC,t,43,2.0,2.0
74109,True,NYC,t,0,0.0,2.0


In [18]:
data_clean_full.columns

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'instant_bookable', 'number_of_reviews', 'bedrooms', 'beds'],
      dtype='object')

In [19]:
data.groupby(["property_type"])["log_price"].mean().sort_values(ascending=False)

property_type
Timeshare             5.602600
Lighthouse            5.521461
Vacation home         5.420951
Castle                5.368424
Earth House           5.285391
Tipi                  5.246118
Train                 5.204399
Boat                  5.200865
Serviced apartment    5.147467
Villa                 5.112427
Boutique hotel        5.083352
Condominium           5.026539
Island                5.010635
Loft                  5.007919
Other                 4.946367
Yurt                  4.944360
Cave                  4.909373
Treehouse             4.845520
Townhouse             4.833539
In-law                4.816649
House                 4.797114
Bungalow              4.783268
Chalet                4.764325
Apartment             4.758850
Guest suite           4.717513
Guesthouse            4.685184
Cabin                 4.646499
Parking Space         4.605170
Camper/RV             4.602238
Bed & Breakfast       4.556068
Casa particular       4.382027
Tent                  4.1

In [20]:
def property_type(prop):
    if prop == "TimeShare":
        return 34
    elif prop == "Lighthouse":
        return 33
    elif prop == "Vacation home":
        return 32
    elif prop == "Castle":
        return 31
    elif prop == "Earth House":
        return 30
    elif prop == "Tipi":
        return 29
    elif prop == "Train":
        return 28
    elif prop == "Boat":
        return 27
    elif prop == "Serviced apartment":
        return 26
    elif prop == "Villa":
        return 25
    elif prop == "Boutique hotel":
        return 24
    elif prop == "Condominium":
        return 23
    elif prop == "Island":
        return 22
    elif prop == "Loft":
        return 21
    elif prop == "Other":
        return 20
    elif prop == "Yurt":
        return 19
    elif prop == "Cave":
        return 18
    elif prop == "Tereehouse":
        return 17
    elif prop == "Townhouse":
        return 16
    elif prop == "In-law":
        return 15
    elif prop == "House":
        return 14
    elif prop == "Bungalow":
        return 13
    elif prop == "Chalet":
        return 12
    elif prop == "Apartment":
        return 11
    elif prop == "Guest suite":
        return 10
    elif prop == "Guesthouse":
        return 9
    elif prop == "Cabin":
        return 8
    elif prop == "Parking space":
        return 7
    elif prop == "Camper/RV":
        return 6
    elif prop == "Bed & Breakfast":
        return 5
    elif prop == "Casa particular":
        return 4
    elif prop == "Tent":
        return 3
    elif prop =="Hut":
        return 2
    elif prop == "Dorm":
        return 1
    else:
        return 0

In [21]:
data_clean_ord = data_clean_full.copy()

In [22]:
data_clean_full = data_clean_full.drop(["city"], axis=1)

In [23]:
data_clean_full["property_type"] = data_clean_full["property_type"].apply(property_type)

In [24]:
data_clean_full

,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,instant_bookable,number_of_reviews,bedrooms,beds
0,5.010635,11,2,9,3,1.0,4,1,True,f,2,1.0,1.0
1,5.129899,11,2,15,7,1.0,4,1,True,t,6,3.0,3.0
2,4.976734,11,2,19,5,1.0,4,2,True,t,10,1.0,3.0
3,6.620073,14,2,15,4,1.0,4,3,True,f,0,2.0,2.0
4,4.744932,11,2,12,2,1.0,4,2,True,t,4,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,11,1,1,1,1.0,4,3,False,f,0,1.0,1.0
74107,5.043425,11,2,16,4,2.0,4,2,True,f,16,2.0,4.0
74108,5.220356,11,2,31,5,1.0,4,2,True,t,43,2.0,2.0
74109,5.273000,11,2,15,2,1.0,4,1,True,t,0,0.0,2.0


In [25]:
data_clean_full['cleaning_fee'] = data_clean_full['cleaning_fee'].map(({True:1,False:0}))
# converting instant_bookable to 0 or 1
data_clean_full['instant_bookable'] = data_clean_full['instant_bookable'].map(({'t':1,'f':0}))

In [26]:
data_clean_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74110 entries, 0 to 74110
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   log_price            74110 non-null  float64
 1   property_type        74110 non-null  int64  
 2   room_type            74110 non-null  int64  
 3   amenities            74110 non-null  int64  
 4   accommodates         74110 non-null  int64  
 5   bathrooms            73910 non-null  float64
 6   bed_type             74110 non-null  int64  
 7   cancellation_policy  74110 non-null  int64  
 8   cleaning_fee         74110 non-null  int64  
 9   instant_bookable     74110 non-null  int64  
 10  number_of_reviews    74110 non-null  int64  
 11  bedrooms             74019 non-null  float64
 12  beds                 73979 non-null  float64
dtypes: float64(4), int64(9)
memory usage: 7.9 MB


In [27]:
y = data_clean_full["log_price"]

In [28]:
data_clean_full = data_clean_full.drop(["log_price"],axis=1)

In [29]:
print(data_clean_full.isna().sum())

property_type            0
room_type                0
amenities                0
accommodates             0
bathrooms              200
bed_type                 0
cancellation_policy      0
cleaning_fee             0
instant_bookable         0
number_of_reviews        0
bedrooms                91
beds                   131
dtype: int64


In [30]:
data_clean_full["bathrooms"].mode()[0]

1.0

In [31]:
data_clean_full["bathrooms"] = data_clean_full["bathrooms"].fillna(data_clean_full["bathrooms"].mode()[0])
data_clean_full["beds"] = data_clean_full["beds"].fillna(data_clean_full["beds"].mode()[0])
data_clean_full["bedrooms"] = data_clean_full["bedrooms"].fillna(data_clean_full["beds"].mode()[0])

In [32]:
data_clean_full.isna().sum()

property_type          0
room_type              0
amenities              0
accommodates           0
bathrooms              0
bed_type               0
cancellation_policy    0
cleaning_fee           0
instant_bookable       0
number_of_reviews      0
bedrooms               0
beds                   0
dtype: int64

In [33]:
X_clean_full = data_clean_full.copy()
X_clean_full

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,instant_bookable,number_of_reviews,bedrooms,beds
0,11,2,9,3,1.0,4,1,1,0,2,1.0,1.0
1,11,2,15,7,1.0,4,1,1,1,6,3.0,3.0
2,11,2,19,5,1.0,4,2,1,1,10,1.0,3.0
3,14,2,15,4,1.0,4,3,1,0,0,2.0,2.0
4,11,2,12,2,1.0,4,2,1,1,4,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
74106,11,1,1,1,1.0,4,3,0,0,0,1.0,1.0
74107,11,2,16,4,2.0,4,2,1,0,16,2.0,4.0
74108,11,2,31,5,1.0,4,2,1,1,43,2.0,2.0
74109,11,2,15,2,1.0,4,1,1,1,0,0.0,2.0


In [34]:
data_clean_ord["city"].value_counts()

NYC        32348
LA         22453
SF          6434
DC          5688
Chicago     3719
Boston      3468
Name: city, dtype: int64

In [35]:
def city_ord(city):
    if city == "San Francisco":
        return 5
    elif city == "DC":
        return 4
    elif city == "Boston":
        return 3
    elif city == "LA":
        return 2
    elif city == "NYC":
        return 1
    else:
        return 0

In [36]:
data_clean_ord["city"] = data_clean_ord["city"].apply(city_ord)

In [37]:
data_clean_ord = data_clean_ord.drop(["property_type"], axis=1)
data_clean_ord['cleaning_fee'] = data_clean_ord['cleaning_fee'].map(({True:1,False:0}))

# converting instant_bookable to 0 or 1
data_clean_ord['instant_bookable'] = data_clean_ord['instant_bookable'].map(({'t':1,'f':0}))

data_clean_ord

,log_price,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,beds
0,5.010635,2,9,3,1.0,4,1,1,1,0,2,1.0,1.0
1,5.129899,2,15,7,1.0,4,1,1,1,1,6,3.0,3.0
2,4.976734,2,19,5,1.0,4,2,1,1,1,10,1.0,3.0
3,6.620073,2,15,4,1.0,4,3,1,0,0,0,2.0,2.0
4,4.744932,2,12,2,1.0,4,2,1,4,1,4,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,1,1,1,1.0,4,3,0,1,0,0,1.0,1.0
74107,5.043425,2,16,4,2.0,4,2,1,2,0,16,2.0,4.0
74108,5.220356,2,31,5,1.0,4,2,1,1,1,43,2.0,2.0
74109,5.273000,2,15,2,1.0,4,1,1,1,1,0,0.0,2.0


In [38]:
data_clean_ord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74110 entries, 0 to 74110
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   log_price            74110 non-null  float64
 1   room_type            74110 non-null  int64  
 2   amenities            74110 non-null  int64  
 3   accommodates         74110 non-null  int64  
 4   bathrooms            73910 non-null  float64
 5   bed_type             74110 non-null  int64  
 6   cancellation_policy  74110 non-null  int64  
 7   cleaning_fee         74110 non-null  int64  
 8   city                 74110 non-null  int64  
 9   instant_bookable     74110 non-null  int64  
 10  number_of_reviews    74110 non-null  int64  
 11  bedrooms             74019 non-null  float64
 12  beds                 73979 non-null  float64
dtypes: float64(4), int64(9)
memory usage: 7.9 MB


In [39]:
y = data_clean_ord["log_price"]
data_clean_ord = data_clean_ord.drop(['log_price'], axis=1)
data_clean_ord["bathrooms"] = data_clean_ord["bathrooms"].fillna(data_clean_ord["bathrooms"].mode()[0])
data_clean_ord["beds"] = data_clean_ord["beds"].fillna(data_clean_ord["beds"].mode()[0])
data_clean_ord["bedrooms"] = data_clean_ord["bedrooms"].fillna(data_clean_ord["beds"].mode()[0])

In [40]:
data_clean_full.columns

Index(['property_type', 'room_type', 'amenities', 'accommodates', 'bathrooms',
       'bed_type', 'cancellation_policy', 'cleaning_fee', 'instant_bookable',
       'number_of_reviews', 'bedrooms', 'beds'],
      dtype='object')

In [41]:
data_clean_full.head()

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,instant_bookable,number_of_reviews,bedrooms,beds
0,11,2,9,3,1.0,4,1,1,0,2,1.0,1.0
1,11,2,15,7,1.0,4,1,1,1,6,3.0,3.0
2,11,2,19,5,1.0,4,2,1,1,10,1.0,3.0
3,14,2,15,4,1.0,4,3,1,0,0,2.0,2.0
4,11,2,12,2,1.0,4,2,1,1,4,0.0,1.0


In [42]:
data_clean_ord.columns

Index(['room_type', 'amenities', 'accommodates', 'bathrooms', 'bed_type',
       'cancellation_policy', 'cleaning_fee', 'city', 'instant_bookable',
       'number_of_reviews', 'bedrooms', 'beds'],
      dtype='object')

In [43]:
data_clean_ord.head()

,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,beds
0,2,9,3,1.0,4,1,1,1,0,2,1.0,1.0
1,2,15,7,1.0,4,1,1,1,1,6,3.0,3.0
2,2,19,5,1.0,4,2,1,1,1,10,1.0,3.0
3,2,15,4,1.0,4,3,1,0,0,0,2.0,2.0
4,2,12,2,1.0,4,2,1,4,1,4,0.0,1.0
